In [2]:
# import talib as ta

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
import seaborn as sns
import matplotlib.pyplot as plt
import csv
#from factor_analyzer import FactorAnalyzer
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.linear_model import LogisticRegression

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 한 실행칸에 프린트 여러개 해도 다 출력시키도록 하는 코드.
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [3]:
raw_data = pd.read_csv('./data.csv')
tech = pd.read_csv('./tech.csv')

FileNotFoundError: [Errno 2] File ./data.csv does not exist: './data.csv'

In [ ]:
raw_data['datetime'] = raw_data['일자'] + ' ' + raw_data['시간']
raw_data['datetime'] = pd.to_datetime(raw_data['datetime'], format='%Y/%m/%d')

In [ ]:
raw_data['datetime'] 

In [ ]:
raw_data = raw_data.iloc[:65284]
tech = tech.iloc[:65284]

In [ ]:
data = raw_data[['datetime', '종가']]
data.columns = ['datetime', 'close']
data = pd.concat([data, tech], axis = 1)
data = data.dropna(axis=0).iloc[:,1:]

In [ ]:
data

In [ ]:
# 120분 평균과 분산
data['mean_120m'] = data['close'].rolling(120).mean()
data['std_120m'] = data['close'].rolling(120).std()
data.dropna(inplace = True)
data.reset_index(inplace = True, drop = True)
# y date list
date_list = [1, 3, 5, 10, 15, 20]
# making y
for i in date_list:
    close_after = data['close'].iloc[i:].reset_index(drop = True)
    nans = np.empty(i)
    nans = np.nan
    close_after = close_after.append(pd.Series(nans, index = np.arange(len(close_after), len(close_after) + i)))
    label =  'y_' + str(i) +'m'
    lower_bound = data['close'] - data['std_120m'] / np.sqrt(120//i)
    upper_bound = data['close'] + data['std_120m'] / np.sqrt(120//i)
    print(data['std_120m'] / np.sqrt(120//i))
    print(close_after)
    print(upper_bound)
    up = close_after >= upper_bound
    down = close_after < lower_bound
    y = 1 * up -1 * down
    data[label] = y
data.drop(['mean_120m', 'std_120m'], axis = 1, inplace = True)

In [ ]:
data['y_1m'].value_counts()

In [ ]:
y_data= data[['y_1m','y_3m','y_5m','y_10m','y_15m','y_20m']]

In [ ]:
data = data.drop(y_data.columns,axis=1)

In [ ]:
pca_selection_data_set = data[:-(396*40)]
classify_predict_data_set = data[-(396*40):]

train_test_index = classify_predict_data_set.index[0]

In [ ]:
"""
    해야 되는 것.
    1. (1~20)분 후의 가격 ~ price_1m
    2. (1~20)분 후의 로그수익률 ~ lgret_1m
    3. (1~20)분 후의 절대수익률 ~ ret_1m
"""
data2 = pca_selection_data_set.copy()
original_col = data2.columns.to_list()

for i in range(1, 21):
    num_label = str(i) + 'm'
    data2['price_' + num_label] = (data2['close'].shift(-i) - data2['close']) * 100
    data2['ret_' + num_label] = ((data2['close'].shift(-i) - data2['close'])/data2['close']) * 100
    data2['logret_' + num_label] = (np.log(data2['close'].shift(-i)/data2['close'])) * 100

In [ ]:
def extract_data(method, period): # method : ('price', 'ret', 'logret'), period = 1~20
    label = method + '_' + str(period) + 'm'
    cols = original_col.copy()
    cols.append(label)
    _data = data2[cols]
    # _data.set_index('datetime', inplace = True, drop = True)
    _data.dropna(inplace = True)
    return _data

In [ ]:
# def lasso_result(x, parameter):
#     all_dict = {}

#     X_train = x.iloc[:,:-1]
#     y_train = x.iloc[:,[-1]]

#     fig, axs = plt.subplots(figsize=(200,200), nrows=1, ncols=len(parameter) )
#     coeff_df = pd.DataFrame()

#      # sns.set(font="AppleGothic", 
#      #        rc={"axes.unicode_minus":False},
#       #       style='darkgrid',font_scale=15)    

#     for pos, al in enumerate(parameter):
#         lasso = Lasso(alpha=al)
#         lasso.fit ( X_train, y_train)
#         print(lasso)
#         coeff = pd.DataFrame(data=lasso.coef_, index=list(X_train.columns),columns=['coeff_value'])
#         coeff['abs_coeff_value'] = abs(coeff['coeff_value'])
#         coeff = coeff.sort_values('abs_coeff_value',ascending=False)
#         colname = 'alpha:'+str(al)
#       #  axs[pos].set_title(colname)
#        # axs[pos].set_xlim(-4,4)  
        
#         coeff_notzero= coeff['coeff_value'][coeff['abs_coeff_value']>=0.01]
#       #  if len(coeff_notzero)>0:
#            # sns.barplot( x=coeff_notzero.values, y=coeff_notzero.index, ax=axs[pos])
#         all_dict[colname] = coeff
#     return(all_dict)

In [ ]:
# def lasso_feature_selection(rank):  
#     list_all = pd.Series()
#     alpha_all = pd.Series()
    
#     alphas = [0.0001, 0.001, 0.01,0.1]
#     for i in ['price', 'ret', 'logret'] :
#         for j in [1,3,5,7,9,11,13,15,17,19]:
#             result_lasso= lasso_result(extract_data(i,j).iloc[:-1],alphas)
#             for k in range(len(list(result_lasso.keys()))):        
#                 result_frame_temp = result_lasso[list(result_lasso.keys())[k]] 
#                 list_temp=pd.Series(result_frame_temp[result_frame_temp['abs_coeff_value']>0.01].iloc[:rank].index)
#                 list_all=list_all.append(list_temp)
#                 alpha_all = alpha_all.append(pd.Series(list(result_lasso.keys())[k]).repeat(len(list_temp)))
#                 print(i)
#                 print(j)
#                 print(list_all.value_counts())
                
#                 print(result_frame_temp[result_frame_temp['abs_coeff_value']>0.01].iloc[:rank])
#     alpha_all.reset_index(inplace=True, drop=True)
#     list_all.reset_index(inplace=True, drop=True)
    
#     final_frame = pd.DataFrame({'feature_all':list_all,'alpha':alpha_all})
#     return(final_frame)

In [ ]:
# result_varaible = lasso_feature_selection(10)

In [ ]:
######여기서부터 돌리기#############

In [ ]:
#선택된 칼럼들
#selected_variable= list(result_varaible['feature_all'].value_counts().index)
selected_variable = pd.read_csv('./lasso_selection_variable.csv')

In [ ]:
selected_variable = selected_variable['0']

In [ ]:
#run all
# selected_variable = data.columns

In [ ]:
#name= 'lasso_selection_variable'
#result_csv = open("./result_"+name+".csv", "a", newline="", encoding="utf8")

#with result_csv:
#    write = csv.writer(result_csv)
#    write.writerows([list(result_varaible['feature_all'].value_counts().index)])

#tick_csv.close()

In [ ]:
#pd.Series(selected_variable).to_csv('./lasso_selection_variable.csv')

In [ ]:
def run_pca_analysis(_train_set, _test_set, _LASSO_columns, threshold, draw_plot):
    
    train_pca_res = train_set_pca_factor(_train_set, threshold, _LASSO_columns, draw_plot)
    
    train_pca_factors = train_pca_res[0]
    train_loadings = train_pca_res[1]
    
    LASSOed_test_set = _test_set[_LASSO_columns]
    test_pca_factors = LASSOed_test_set @ train_loadings
    
    
    
    return train_pca_factors, train_loadings, test_pca_factors

In [ ]:
# X = LASSO로 의미없는 변수 제거한 DataFrame

def train_set_pca_factor(_train_set, threshold, _LASSO_columns, draw_plot = False):
    
    LASSOed_train_set = _train_set[_LASSO_columns]
    max_pca_factor = LASSOed_train_set.shape[1]
    LASSOed_train_set = StandardScaler().fit_transform(LASSOed_train_set)
    pca = PCA(n_components = max_pca_factor)
    principalComponents = pca.fit_transform(LASSOed_train_set)
    explain_var = pca.explained_variance_ratio_
    num_pca_factor = len(explain_var[explain_var.cumsum() < threshold])
    print("PCA threshold 넘기기 위해 필요한 최소 Factor: ",num_pca_factor)
    
    if draw_plot == True:
        fig, axes = plt.subplots(nrows=2, ncols=1)

        axes[0].plot(explain_var.cumsum(), label = "explained_Var")
        axes[1].set_xlabel("# of PCA Factors")
        axes[0].set_ylabel("cum_explained_variance")
        axes[0].set_title("PCA_explained_variance")
        axes[0].axhline(y=0.7, color='r', linewidth=1, label = "dd")
        axes[0].legend(["explained_Var", "Threshold"])


        axes[1].bar(np.arange(max_pca_factor) , explain_var)
        axes[1].set_ylabel("explained_variance")
    
    pca = PCA(n_components = num_pca_factor)
    principalComponents = pca.fit_transform(LASSOed_train_set)
    train_set_pca_factors = pd.DataFrame(data=principalComponents, 
                                         columns=["PC"+str(i+1) for i in range(num_pca_factor)])
    
    _LASSO_columns = np.array(_LASSO_columns)
    train_set_loadings = pd.DataFrame(pca.components_.T,
                        columns=["PC"+str(i+1) for i in range(num_pca_factor)],
                        index=np.array(_LASSO_columns).T)
    
    return train_set_pca_factors, train_set_loadings

In [ ]:
window_size = 396 # 1 day data
train_size = window_size * 5
test_size = window_size

result_dict = {}

for j in range(len(date_list)):
    y_label = "y_" + str(date_list[j]) + "m"
    y = y_data.loc[train_test_index:, y_label]
    i = train_test_index
    test_results = pd.DataFrame(columns=['act_'+str(date_list[j]),'pred_'+str(date_list[j])])
    while(i + train_size + test_size <= len(data)):
        print(y_label,raw_data['datetime'].loc[i + train_size + test_size] )
        print(raw_data['datetime'].loc[len(data)])
        
        # PCA func (with YK's func)
        _, _, sub_data = run_pca_analysis(pca_selection_data_set, 
                                          classify_predict_data_set.loc[i:i+train_size + test_size] ,
                                          selected_variable, 0.7, draw_plot = True )
        
 
           # 데이터 쪼개기 : train & test data
        train = pd.concat([sub_data.loc[i : i + train_size], y.loc[i : i + train_size]], axis=1)
        test = pd.concat([sub_data.loc[i+train_size : i+train_size + test_size], y.loc[i+train_size : i+train_size + test_size]], axis=1)
        
        # train : like logistic regression(with softmax)
        
        softmax_reg = LogisticRegression(multi_class="multinomial", penalty='l2', C=10, random_state=42)
        softmax_reg.fit(train.iloc[:,:-1], train.iloc[:, -1])
        
        
        # test : 
        res = softmax_reg.predict(test.iloc[:,:-1])
        
        test_results = pd.concat([test_results, 
                                  pd.DataFrame({'act_'+str(date_list[j]):test.iloc[:, -1],
                                                'pred_'+str(date_list[j]):res})])
        
        i = i + window_size
        
    result_dict[y_label] = test_results # {'y_1m' : [], 'y_3m' : []} []은 y & yhat

## Accuracy

In [ ]:
act = result_dict['y_10m']['act_10'].to_list()
pred = result_dict['y_10m']['pred_10'].to_list()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
accuracy_metric = accuracy_score(act, pred)

In [ ]:
accuracy_metric

In [ ]:
print(f1_score(act, pred, average="macro"))
print(precision_score(act, pred, average="macro"))
print(recall_score(act, pred, average="macro"))

In [ ]:
confusion = confusion_matrix(act, pred)
confusion

In [ ]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(act, pred))

## Trading Profit Evaluation

In [ ]:
##필요한 DataSet
predict_res_dict= result_dict.copy()
temp_d = classify_predict_data_set.copy()
original_col = temp_d.columns.to_list()

##Label 생성
for i in range(1, 21):
    num_label = str(i) + 'm'
    
    #price change times 100!
    temp_d['price_' + num_label] = (temp_d['close'].shift(-i) - temp_d['close']) * 100
    
    #만원단위 Tick Value(Lead로 계산하기위해 shift(-i))
    temp_d['tick_value_' + num_label] = (temp_d['close'].shift(-i) - temp_d['close'])/0.05*2.5
    
    temp_d['ret_' + num_label] = ((temp_d['close'].shift(-i) - temp_d['close'])/temp_d['close']) * 100
    temp_d['logret_' + num_label] = (np.log(temp_d['close'].shift(-i)/temp_d['close'])) * 100

In [ ]:
temp_d

In [ ]:
trading_profit_RESULT = pd.DataFrame()

##검증 필요
y = [1, 3, 5, 10, 15, 20]

for i in y:
    profit_df = pd.merge(temp_d["tick_value_"+str(i)+"m"],result_dict["y_"+str(i)+"m"]["pred_"+str(i)],
                         left_index= True, right_index=True, how ="left")
    #앞: 5일치 train 데이터 빠짐 
    #뒤: lead 계산한 만큼 빠짐
#     profit_df = profit_df.dropna()
    
    
    for row in range(len(profit_df)):
        if profit_df["pred_"+str(i)].iloc[row] == 1 or profit_df["pred_"+str(i)].iloc[row] == -1:
            profit_df["pred_"+str(i)].iloc[row+1:row+5] = 0
            
#         elif profit_df["pred_"+str(i)].iloc[row] == -1:
#             profit_df["pred_"+str(i)].iloc[row+1:row+5] = 0

        else:
            pass
        
    
    profit_df["profit"] = profit_df["tick_value_"+str(i)+"m"] * profit_df["pred_"+str(i)]
    trading_profit_RESULT[str(i)+"m_cum_profit"] = profit_df["profit"].cumsum()
    
trading_profit_RESULT.dropna(how = "all")
    

In [ ]:
trading_profit_RESULT["1m_cum_profit"].plot()

In [ ]:
trading_profit_RESULT["3m_cum_profit"].plot()

In [ ]:
trading_profit_RESULT["5m_cum_profit"].plot()

In [ ]:
trading_profit_RESULT["10m_cum_profit"].plot()

In [ ]:
trading_profit_RESULT["15m_cum_profit"].plot()

In [ ]:
trading_profit_RESULT["20m_cum_profit"].plot()

In [ ]:
trading_profit_RESULT.plot()